# STM Transit Delay Data Preparation

## Overview

This notebook merges data from multiple sources and prepares it for data analysis and/or preprocessing.

## Data description

### Real-time Trip Updates

`current_time`: Timestamp when the data was fetched from the GTFS, in microseconds.<br>
`trip_id`: Unique identifier for the transit trip.<br>
`route_id`: Unique identifier for a bus or metro line.<br>
`start_date`: Start date of the transit trip.<br>
`stop_id`: Unique identifier of a stop.<br>
`arrival_time`, `departure_time`: Realtime arrival and departure time, in microseconds<br>
`schedule_relationship`: State of the trip, 0 meaning "scheduled", 1 meaning "skipped" and 2 meaning "no data".

### Scheduled STM Trips

`trip_id`: Unique identifier for the transit trip.<br>
`arrival_time`, `departure_time`: Scheduled arrival and departure time.<br>
`stop_id`: Unique identifier of a stop.<br>
`stop_sequence`: Sequence of a stop, for ordering.

### STM Stops

`stop_id`: Unique identifier of a stop.<br>
`stop_code`: Bus stop or metro station number.<br>
`stop_name`: Bus stop or metro station name<br>
`stop_lat`, `stop_lon`: Stop coordinates.<br>
`stop_url`: Stop web page.<br>
`location_type`: Stop type.<br>
`parent_station`: Parent station (metro station with multiple exits).<br>
`wheelchair_boarding`: Indicates if the stop is accessible for people in wheelchair, 1 being true and 2 being false.

### Real-time Vehicle Positions

`current_time`: Timestamp when the data was fetched from the GTFS, in microseconds.<br>
`vehicle_id`: Unique identifier for a vehicle.<br>
`trip_id`: Unique identifier for the transit trip.<br>
`route_id`: Unique identifier for a bus or metro line.<br>
`start_date`: Start date of a transit trip.<br>
`start_time`: Start time of a transit trip.<br>
`latitude`, `longitude`: Vehicle current position.<br>
`bearing`: Direction that the vehicle is facing, from 0 to 360 degrees.<br>
`speed`: Momentary speed measured by the vehicle, in meters per second.<br>
`stop_sequence`: Sequence of a stop, for ordering.<br>
`status`: Vehicle stop status in relation with a stop that it's currently approaching or is at, 1 being "stopped at" and 2 being "in transit to".<br>
`timestamp`: Timestamp when STM updated the data, in milliseconds.<br>
`occupancy_status`: Degree of passenger occupancy, ranging from 1 (empty) to 7 (not accepting passengers).

### Service Alerts

`start_time`: Start time of the alert, in microseconds.<br>
`end_time`: End time of the alert, in microseconds.<br>
`route_id`: Unique identifier for a bus or metro line.<br>
`stop_id`: Unique identifier of a stop.

### Weather Archive and Forecast

`time`: Date and hour or the weather.<br>
`temperature`: Air temperature at 2 meters above ground, in Celsius.<br>
`precipitation`: Total precipitation (rain, showers, snow) sum of the preceding hour, in millimeters.<br>
`windspeed`: Wind speed at 10 meters above ground, in kilometers per hour.<br>
`weathercode`: World Meteorological Organization (WMO) code, see [Open-Meteo API documentation](https://open-meteo.com/en/docs#weather_variable_documentation) for the list.

### Traffic Incidents

`category`: Category of the incident.<br>
`start_time`: Start time of the incident, in ISO8601 format.<br>
`end_time`: End time of the incident, in ISO8601 format.<br>
`length`: Length of the incident in meters.<br>
`delay`: Delay in seconds caused by the incident (except road closures).<br>
`magnitude_of_delay`: Severity of the delay, ranging from 0 to 4 (minor to major).<br>
`last_report_time`: Date when the last time the incident was reported,in ISO8601 format.<br>
`latitude`, `longitude`: Coordinates of the incident.

## Imports

In [ ]:
from datetime import datetime, timedelta, timezone
from haversine import haversine, Unit
import numpy as np
import pandas as pd
import sys

In [ ]:
# Import custom code
sys.path.insert(0, '..')
from scripts.custom_functions import fetch_weather, LOCAL_TIMEZONE, OCCUPANCY_STATUS, SCHEDULE_RELATIONSHIP, WEATHER_CODES

In [ ]:
# Import data
trips_df = pd.read_csv('../data/api/fetched_stm_trip_updates.csv', low_memory=False)
schedules_df = pd.read_csv('../data/download/stop_times_2025-04-30.txt')
stops_df = pd.read_csv('../data/download/stops_2025-04-30.txt')
positions_df =  pd.read_csv('../data/api/fetched_stm_vehicle_positions.csv', low_memory=False)
alerts_df = pd.read_csv('../data/api/fetched_stm_service_alerts.csv')
weather_df = pd.read_csv('../data/api/fetched_historical_weather.csv')
traffic_df = pd.read_csv('../data/api/fetched_traffic.csv')

## Merge Data

### Schedules and stops

In [ ]:
# Sort values by stop sequence
schedules_df = schedules_df.sort_values(by=['trip_id', 'stop_sequence'])

In [ ]:
# Add trip progress (vehicles further along the trip are more likely to be delayed)
total_stops = schedules_df.groupby('trip_id')['stop_id'].transform('count')
schedules_df['trip_progress'] = schedules_df['stop_sequence'] / total_stops

In [ ]:
# Get distribution of trip progress
schedules_df['trip_progress'].describe()

In [ ]:
# Merge schedules and stops
schedules_stops_df = pd.merge(left=schedules_df, right=stops_df, how='inner', left_on='stop_id', right_on='stop_code')

In [ ]:
schedules_stops_df.columns

In [ ]:
# Rename stop id and drop other stop id columns
schedules_stops_df = schedules_stops_df.rename(columns={'stop_id_x': 'stop_id'})
schedules_stops_df = schedules_stops_df.drop(['stop_id_y', 'stop_code'], axis=1)

In [ ]:
# Get coordinates of previous stop
schedules_stops_df = schedules_stops_df.sort_values(by=['trip_id', 'stop_sequence'])
schedules_stops_df['prev_lat'] = schedules_stops_df.groupby('trip_id')['stop_lat'].shift(1)
schedules_stops_df['prev_lon'] = schedules_stops_df.groupby('trip_id')['stop_lon'].shift(1)

In [ ]:
# Make sure the null values are from first stops
prev_null_mask = (schedules_stops_df['prev_lat'].isna()) | (schedules_stops_df['prev_lon'].isna())
first_stop_mask = schedules_stops_df['stop_sequence'] == 1
assert prev_null_mask.sum() == first_stop_mask.sum()

In [ ]:
# Get distance from previous stop
schedules_stops_df['stop_distance'] = schedules_stops_df.apply(
  lambda row: haversine((row['prev_lat'], row['prev_lon']), (row['stop_lat'], row['stop_lon']), unit=Unit.METERS),
  axis=1
)

In [ ]:
# Drop previous coordinates
schedules_stops_df = schedules_stops_df.drop(['prev_lat', 'prev_lon'], axis=1)

In [ ]:
# Replace null distances by zero (first stop of the trip)
schedules_stops_df['stop_distance'] = schedules_stops_df['stop_distance'].fillna(0)

In [ ]:
schedules_stops_df['stop_distance'].describe()

In [ ]:
# Get stop with largest distance
schedules_stops_df.iloc[schedules_stops_df['stop_distance'].idxmax()]

After double-checking in the STM website, the large distance make sense because the bus stops before or after this are far away.

### Realtime and Scheduled Trips

In [ ]:
# Convert route_id to integer
trips_df['route_id'] = trips_df['route_id'].str.extract(r'(\d+)')
trips_df['route_id'] = trips_df['route_id'].astype('int64')

In [ ]:
# Get proportion of duplicates
subset = trips_df.drop('current_time', axis=1).columns
duplicate_mask = trips_df.duplicated(subset=subset)
print(f'{duplicate_mask.mean():.2%}')

In [ ]:
# Remove duplicates
trips_df = trips_df.drop_duplicates(subset=subset)

In [ ]:
# Rename arrival and departure time
trips_df = trips_df.rename(columns={'arrival_time': 'rt_arrival_time','departure_time': 'rt_departure_time'})

In [ ]:
# Merge trip updates with schedule
stm_trips_df = pd.merge(left=trips_df, right=schedules_stops_df, how='inner', on=['trip_id', 'stop_id'])

In [ ]:
stm_trips_df.columns

In [ ]:
# Convert start_date to datetime
stm_trips_df['start_date_dt'] = pd.to_datetime(stm_trips_df['start_date'], format='%Y%m%d')

In [ ]:
def parse_gtfs_time(start_date:pd.Timestamp, stop_time:str) -> pd.Timestamp:
	'''
	Converts GTFS time string (e.g., '25:30:00') to datetime
	based on the arrival time.
	'''
	hours, minutes, seconds = map(int, stop_time.split(':'))
	total_seconds = hours * 3600 + minutes * 60 + seconds

	parsed_time = start_date + timedelta(seconds=total_seconds)
	return parsed_time

In [ ]:
# Parse GTFS scheduled arrival and departure time
parsed_arrival_time = stm_trips_df.apply(lambda row: parse_gtfs_time(row['start_date_dt'], row['arrival_time']), axis=1)
parsed_departure_time = stm_trips_df.apply(lambda row: parse_gtfs_time(row['start_date_dt'], row['departure_time']), axis=1)

In [ ]:
# Convert scheduled arrival and departure time to UTC datetime
sch_arrival_time_local = parsed_arrival_time.dt.tz_localize(LOCAL_TIMEZONE)
sch_departure_time_local = parsed_departure_time.dt.tz_localize(LOCAL_TIMEZONE)

stm_trips_df['sch_arrival_time'] = sch_arrival_time_local.dt.tz_convert(timezone.utc)
stm_trips_df['sch_departure_time'] = sch_departure_time_local.dt.tz_convert(timezone.utc)

In [ ]:
# Get rows where scheduled arrival and departure time are different
stm_trips_df[stm_trips_df['sch_arrival_time'] != stm_trips_df['sch_departure_time']]

In [ ]:
# Replace 0 timestamps with NaN
stm_trips_df['rt_arrival_time'] = stm_trips_df['rt_arrival_time'].replace({0: np.nan})
stm_trips_df['rt_departure_time'] = stm_trips_df['rt_departure_time'].replace({0: np.nan})

In [ ]:
# Get distribution of timestamps
stm_trips_df[['rt_arrival_time', 'rt_departure_time']].describe()

In [ ]:
# Convert realtime arrival and departure time to UTC datetime
stm_trips_df['rt_arrival_time'] = pd.to_datetime(stm_trips_df['rt_arrival_time'] * 1000, origin='unix', unit='ms', utc=True)
stm_trips_df['rt_departure_time'] = pd.to_datetime(stm_trips_df['rt_departure_time'] * 1000, origin='unix', unit='ms', utc=True)

In [ ]:
# Calculate delay (realtime - scheduled)
# For first stops, calculate with departure time
# For the rest, calculate with arrival time
stm_trips_df['delay'] = np.where(
  	stm_trips_df['stop_sequence'] == 1,
	(stm_trips_df['rt_departure_time'] - stm_trips_df['sch_departure_time']) / pd.Timedelta(seconds=1),
  	(stm_trips_df['rt_arrival_time'] - stm_trips_df['sch_arrival_time']) / pd.Timedelta(seconds=1)
)

In [ ]:
# Get distribution
stm_trips_df['delay'].describe()

In [ ]:
# Get null delays
print(stm_trips_df['delay'].isna().sum())

In [ ]:
# Replace the null delays with the average delay by route
stm_trips_df['delay'] = stm_trips_df['delay'].fillna(stm_trips_df.groupby(['route_id'])['delay'].transform('mean'))

In [ ]:
assert stm_trips_df['delay'].isna().sum() == 0

In [ ]:
stm_trips_df.columns

In [ ]:
# Remove extra datetime columns
stm_trips_df = stm_trips_df.drop(['arrival_time', 'departure_time', 'start_date_dt'], axis=1)

### Vehicle Positions

In [ ]:
# Get proportion of duplicates
subset = positions_df.drop('current_time', axis=1).columns
duplicate_mask = positions_df.duplicated(subset=subset)
print(f'{duplicate_mask.mean():.2%}')

In [ ]:
# Drop duplicates
positions_df = positions_df.drop_duplicates(subset=subset)

In [ ]:
# Rename latitude and longitude
positions_df = positions_df.rename(columns={
  'latitude': 'vehicle_lat',
  'longitude': 'vehicle_lon',
  'status': 'vehicle_status',
  'bearing': 'vehicle_bearing',
  'speed': 'vehicle_speed',
  'timestamp': 'vehicle_dt'
})

In [ ]:
# Merge positions
stm_trips_positions_df = pd.merge(left=stm_trips_df, right=positions_df, how='inner', on=['trip_id', 'route_id', 'start_date', 'stop_sequence'])

In [ ]:
stm_trips_positions_df.columns

In [ ]:
# Drop current timestamps and start_date
stm_trips_positions_df = stm_trips_positions_df.drop(['current_time_x', 'current_time_y', 'start_date'], axis=1)

In [ ]:
# Calculate distance between the vehicle and the current stop
stm_trips_positions_df['vehicle_distance'] = stm_trips_positions_df.apply(
  	lambda row: haversine((row['vehicle_lat'], row['vehicle_lon']), (row['stop_lat'], row['stop_lon']), unit=Unit.METERS),
  	axis=1)

In [ ]:
stm_trips_positions_df['vehicle_distance'].describe()

In [ ]:
further_than_previous_stop = stm_trips_positions_df['vehicle_distance'] > stm_trips_positions_df['stop_distance']
stm_trips_positions_df[further_than_previous_stop]

The large vehicle distances don't make sense with the delays. It will confuse the model so the coordinates and the distance won't be used.

In [ ]:
# Drop unneeded columns
stm_trips_positions_df = stm_trips_positions_df.drop(['vehicle_lat', 'vehicle_lon', 'vehicle_dt', 'vehicle_distance', 'start_time'], axis=1)

### Service Alerts

In [ ]:
# Get proportion of duplicates
duplicate_mask = alerts_df.duplicated()
print(f'{duplicate_mask.mean():.2%}')

In [ ]:
# Remove duplicates
alerts_df = alerts_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Convert timestamps to datetime
alerts_df['start_time'] = pd.to_datetime(alerts_df['start_time'] * 1000, origin='unix', unit='ms', utc=True)
alerts_df['end_time'] = pd.to_datetime(alerts_df['end_time'] * 1000, origin='unix', unit='ms', utc=True)

In [ ]:
# Fill null end time with current date (assuming the alert is still active)
alerts_df['end_time'] = alerts_df['end_time'].fillna(datetime.now(timezone.utc).replace(microsecond=0))

In [ ]:
# Sort values by date
alerts_df = alerts_df.sort_values('start_time').reset_index()

In [ ]:
stm_df = pd.merge(left=stm_trips_positions_df, right=alerts_df, how='left', on=['route_id', 'stop_id'])

In [ ]:
stm_df.columns

In [ ]:
# Add column has_alert
has_alert_mask = (stm_df['start_time'].notna()) & \
	(stm_df['rt_arrival_time'] >= stm_df['start_time']) & \
	(stm_df['rt_arrival_time'] <= stm_df['end_time'])
stm_df['stop_has_alert'] = has_alert_mask.astype('int64')

In [ ]:
stm_df['stop_has_alert'].value_counts()

In [ ]:
# Drop unneeded datetime columns and index
stm_df = stm_df.drop(['start_time', 'end_time', 'index'], axis=1)

### STM and Weather

In [ ]:
# Convert time string to datetime
time_dt = pd.to_datetime(weather_df['time'], utc=True)

In [ ]:
# Calculate dates for weather forecast
last_day_weather = time_dt.max()
start_date = last_day_weather + timedelta(days=1)
end_date = stm_df['rt_arrival_time'].max()

In [ ]:
# Fetch forecast weather
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

forecast_list = fetch_weather(start_date=start_date_str, end_date=end_date_str, forecast=True)
forecast_df = pd.DataFrame(forecast_list)

In [ ]:
# Merge archive and forecast weather
weather_df = pd.concat([weather_df, forecast_df], ignore_index=True)

In [ ]:
# Round arrival time to the nearest hour
rounded_arrival_dt = stm_df['rt_arrival_time'].dt.round('h')

In [ ]:
# Format time to match weather data
stm_df['time'] = rounded_arrival_dt.dt.strftime('%Y-%m-%dT%H:%M')

In [ ]:
# Merge STM with weather
stm_weather_df = pd.merge(left=stm_df, right=weather_df, how='inner', on='time').drop('time', axis=1)

### Traffic Data

In [ ]:
# Get proportion of duplicates
duplicate_mask = traffic_df.duplicated()
print(f'{duplicate_mask.mean():.2%}')

In [ ]:
# Remove duplicates
traffic_df = traffic_df.drop_duplicates(keep='last').reset_index()

In [ ]:
# Convert traffic start_time and end_time to datetime
traffic_df['start_time'] = pd.to_datetime(traffic_df['start_time'], utc=True)
traffic_df['end_time'] = pd.to_datetime(traffic_df['end_time'], utc=True)

In [ ]:
# Sort by date
traffic_df = traffic_df.sort_values(by='start_time').reset_index()

In [ ]:
# Fill null end times with current time (assuming the incident is still ongoing)
traffic_df['end_time'] = traffic_df['end_time'].fillna(datetime.now(timezone.utc).replace(microsecond=0))
assert traffic_df['end_time'].isna().sum() == 0

In [ ]:
# Build traffic cache (for every 15 min interval)
def build_traffic_cache(traffic_df:pd.DataFrame) -> dict:
	traffic_cache = {}
	traffic_df['quarter_hour'] = traffic_df['start_time'].dt.floor('15min')

	for (hour, group) in traffic_df.groupby('quarter_hour'):
		traffic_cache[hour] = group.copy()

	return traffic_cache

Since there are many trip updates on the same day (even the same hour), there's a risk of repeating the filtering of active traffic incidents for each trip individually, which takes a lot of time for a large dataset. Traffic incidents are stable over minutes or hours. This is why the incidents will be cached by 15 minute intervals.

In [ ]:
def calculate_nearby_incidents(trip_update:pd.Series, traffic_cache:dict, max_distance:int=500) -> pd.Series:
	trip_datetime = trip_update['rt_arrival_time']
	stop_coords = (trip_update['stop_lat'], trip_update['stop_lon'])

	trip_quarter_hour = trip_datetime.floor('15min')

	# Get cached incidents
	quarter_hour_incidents = traffic_cache.get(trip_quarter_hour)

	# Stop if there are no incidents for that hour
	if quarter_hour_incidents is None or quarter_hour_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})

	# Filter for active incidents at that trip hour
	active_incidents = quarter_hour_incidents[
		(quarter_hour_incidents['start_time'] <= trip_datetime) &
		(quarter_hour_incidents['end_time'] >= trip_datetime)
	].copy()

	if active_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})

	# Calculate distance     
	active_incidents['distance'] = active_incidents.apply(
		lambda row: haversine(stop_coords, (row['latitude'], row['longitude']), unit=Unit.METERS),
		axis=1
	)

	# Filter nearby
	nearby_incidents = active_incidents[active_incidents['distance'] <= max_distance]

	if nearby_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})
	else:
		nearest = nearby_incidents.loc[nearby_incidents['distance'].idxmin()]
		return pd.Series({
			'incident_nearby': 1,
			'nearest_incident_distance': nearest['distance'],
			'incident_category': nearest['category'],
			'incident_delay': nearest['delay'],
			'incident_delay_magnitude': nearest['magnitude_of_delay']
		})

In [ ]:
# Get traffic columns (get incidents within 2 km)
traffic_cache = build_traffic_cache(traffic_df)
traffic_cols = stm_weather_df.apply(lambda row: calculate_nearby_incidents(row, traffic_cache, 2000), axis=1)

General area traffic (within 1-2 km) could affect delays, even if it's not directly at the stop. This is why incidents within 2 km are calculated.

In [ ]:
# Merge the traffic
df = pd.concat([stm_weather_df, traffic_cols], axis=1)

## Clean Data

### Convert columns

In [ ]:
# Get columns with two values
two_values = df.loc[:, df.nunique() == 2]
two_values.columns

In [ ]:
print(df['wheelchair_boarding'].value_counts())
print(df['vehicle_status'].value_counts())
print(df['incident_nearby'].value_counts())
print(df['stop_has_alert'].value_counts())

In [ ]:
# Convert columns with 2 unique values to integer
df['wheelchair_boarding'] = (df['wheelchair_boarding'] == 1).astype('int64')
df['vehicle_in_transit'] = (df['vehicle_status'] == 2).astype('int64')
df['incident_nearby'] = df['incident_nearby'].astype('int64')

### Drop columns

In [ ]:
# Remove columns with constant values or with more than 50% missing values
df = df.loc[:, (df.nunique() > 1) & (df.isna().mean() < 0.5)]
df.columns

**Other columns to drop**

`stop_url`: An url is not useful for data analysis.<br>
`vehicle_status`: It has been converted to vehicle_in_transit.<br>

### Create Delay Categories

In [ ]:
df['delay'].describe()

In [ ]:
# Create ranges and labels based on distribution
labels = ['Very Early', 'Early', 'On Time', 'Late', 'Very Late']
ranges = [-np.inf, -60, 0, 60, 300, np.inf]

In [ ]:
# Add delay category column
df['delay_class'] = pd.cut(df['delay'], bins=ranges, labels=labels, include_lowest=True, right=False)
df['delay_class'].value_counts(normalize=True)

### Create Weather Categories

In [ ]:
# Create weather code mapping
weathercodes = df['weathercode'].sort_values().unique()
condition_list = []
label_list = []

for code in weathercodes:
  condition_list.append(df['weathercode'] == code)
  label_list.append(WEATHER_CODES[code])

In [ ]:
# Create categories
df['weather'] = np.select(condition_list, label_list, default='Unknown')
df['weather'].value_counts()

### Convert Schedule Relationship to Categories

In [ ]:
# Create schedule relationship mapping
sch_codes = df['schedule_relationship'].sort_values().unique()
condition_list = []
label_list = []

for code in sch_codes:
  condition_list.append(df['schedule_relationship'] == code)
  label_list.append(SCHEDULE_RELATIONSHIP[code])

In [ ]:
# Create categories
df['schedule_relationship'] = np.select(condition_list, label_list, default='Unknown')
df['schedule_relationship'].value_counts()

### Convert Occupancy Status to Categories

In [ ]:
# Create occupancy status mapping
occ_codes = df['occupancy_status'].sort_values().unique()
condition_list = []
label_list = []

for code in occ_codes:
  condition_list.append(df['occupancy_status'] == code)
  label_list.append(OCCUPANCY_STATUS[code])

In [ ]:
# Create categories
df['occupancy_status'] = np.select(condition_list, label_list, default='Unknown')
df['occupancy_status'].value_counts()

## Export Data

In [ ]:
# Drop and reorder columns
df = df[[
	'trip_id',
  	'vehicle_id',
    'vehicle_in_transit', # converted
    'vehicle_bearing',
    'vehicle_speed',
	'occupancy_status',
  	'route_id',
  	'stop_id',
    'stop_name',
  	'stop_lat',
  	'stop_lon',
    'stop_distance', # engineered
    'stop_sequence',
  	'trip_progress', # engineered
    'stop_has_alert', # engineered
  	'wheelchair_boarding', # engineered
    'schedule_relationship',
  	'rt_arrival_time', # parsed
    'sch_arrival_time', # parsed
    'rt_departure_time', # parsed
    'sch_departure_time', # parsed
    'delay', # engineered
    'delay_class', # engineered
  	'temperature',
  	'precipitation',
  	'windspeed', 
	'weather',
  	'incident_nearby', # engineered
]]

In [ ]:
# Export data to CSV
df.to_csv('../data/stm_weather_traffic_merged.csv', index=False)

## End